In [ ]:
!pip3 install keras-visualizer
!pip3 install livelossplot

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import pickle

from google.colab import drive

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from IPython.display import Image
from livelossplot import PlotLossesKeras

import tensorflow as tf
from tensorflow.keras import Sequential,models,layers
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Activation
from keras_visualizer import visualizer 

In [ ]:
drive.mount('/content/drive')

In [ ]:
dataset = '/content/drive/MyDrive/Colab Notebooks/dataset/adult.csv'
headers = ['age', 	'workclass' , 	'education' ,	'marital.status' ,	'occupation', 	'sex' ,	'hours.per.week' ,	'native.country', 	'income']
ds = pd.read_csv(dataset , usecols=headers)
ds.head()

Fix missing data

In [ ]:
# print(ds[ds == '?'].value_counts())

In [ ]:
imputer = SimpleImputer(missing_values='?', strategy='most_frequent')
imputer = imputer.fit(ds)
imputedataset = imputer.transform(ds)

df = pd.DataFrame(imputedataset,columns=headers).astype(ds.dtypes.to_dict())
df.head()

Fix Bias

In [ ]:
print(df['income'].value_counts())
print('ratio:',df['income'].value_counts()[0] / df['income'].value_counts()[1])

In [ ]:
data_sel =  df[df['income'] == '>50K']
new_df = pd.concat([data_sel] * 3,ignore_index=True)

df = new_df.append(df[df['income'] == '<=50K'],ignore_index=True)

In [ ]:
print(df['income'].value_counts())
# print('ratio:',new_df['income'].value_counts()[0] / new_df['income'].value_counts()[0])

In [ ]:
df.info()

LabelEncode on categoriacal data

In [ ]:
le = LabelEncoder()

for columns in df.columns[df.dtypes == 'object']:
  df[columns] = le.fit_transform(df[columns])


In [ ]:
df.head()

X Y separate and traintest split

In [ ]:
x_data = df.drop(columns=['income'])
y_data = df['income']

In [ ]:
x_data

In [ ]:
y_data

In [ ]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [ ]:
x_data

In [ ]:
y_data

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size=.33,random_state=1)

Scale data

In [ ]:
# scale = StandardScaler()
# x_train = scale.fit_transform(x_train)
# x_test = scale.fit_transform(x_test)

Artifical intelligence 

In [ ]:
# Define the model
def build_model(optimizer='adam'):
  model = Sequential()
  model.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  return model

# Create a pipeline to scale the data and build the model
pipeline = Pipeline([
  ('scaler', StandardScaler()),
  ('model', KerasClassifier(build_fn=build_model, epochs=20, batch_size=32, verbose=3))
])

# Define the grid search parameters
param_grid = {
  'model__optimizer': ['adam', 'sgd']
}

# Create the grid search object
grid_search = GridSearchCV(pipeline, param_grid, cv=10, verbose=2)

# Fit the grid search object to the training data
grid_search.fit(x_train, y_train)


In [ ]:
print(f'Best estimator: {grid_search.best_estimator_}')
print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_:.3f}')

In [ ]:
# y_pred = grid_search
grid_search.score(x_test,y_test)

In [ ]:
best_prediction = grid_search.predict(x_test)

In [ ]:
print(classification_report(y_test,best_prediction))

PREDICTION

In [ ]:
inputdata = np.array([[90,'Private', '12th' , 'Never-married' , 'Armed-Forces' , 'Male', 60, 'India', '>50k']])
inputhead = ['age','workclass','education','marital.status','occupation','sex','hours.per.week','native.country','income']

dict = {
    'age': int,
    'workclass':str,
    'education':str,
    'marital.status':str,
    'occupation':str,
    'sex':str,
    'hours.per.week':int,
    'native.country':str
}

testdataframe = pd.DataFrame(inputdata,columns=inputhead).astype(dict)
testdataframe.head()

In [ ]:
# for columns in testdataframe.columns[testdataframe.dtypes == 'object']:
#   testdataframe[columns] = le.transform(testdataframe[columns])
enc = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/labelencoeder.pkl', "rb"))
testdataframe['workclass'] = enc.transform(testdataframe['workclass'])
testdataframe.head()

In [ ]:
grid_search.predict(x_test)

save

In [ ]:
pickle.dump(le,open("/content/drive/MyDrive/Colab Notebooks/labelencoeder.pkl", "wb"))
pickle.dump(pipeline,open("/content/drive/MyDrive/Colab Notebooks/pipeline.pkl", "wb"))
pickle.dump(grid_search,open("/content/drive/MyDrive/Colab Notebooks/grid_search.pkl", "wb"))

pickle.dump(grid_search,open("/content/drive/MyDrive/Colab Notebooks/actual_model.pkl", "wb"))